In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [22]:
data = pd.read_csv("./data/properatti.csv", index_col=0)
data.shape

(121220, 25)

In [23]:
# grupos de columas para poder trabajar con ellas de forma mas sencilla
geolocation = ['state_name','geonames_id', 'lat-lon','lat','lon']

# Creamos un nuevo dataframe con las columnas que me interesan

geo_location_data = (data[ geolocation].copy())
geo_location_data


,state_name,geonames_id,lat-lon,lat,lon
0,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839
1,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,-57.964330
2,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,-58.522982
3,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,-58.516424
4,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,-57.549447
...,...,...,...,...,...
121215,Capital Federal,3436077.0,NaN,NaN,NaN
121216,Bs.As. G.B.A. Zona Norte,3436080.0,NaN,NaN,NaN
121217,Capital Federal,3433775.0,"-34.5706388726,-58.4755963355",-34.570639,-58.475596
121218,Buenos Aires Costa Atlántica,NaN,NaN,NaN,NaN


In [24]:
masc_nulos_lat = geo_location_data["lat"].astype(str)  == "nan"
geo_location_data[masc_nulos_lat].index

Int64Index([    13,     14,     23,     42,     49,     62,     86,     98,
               104,    105,
            ...
            121208, 121210, 121211, 121212, 121213, 121214, 121215, 121216,
            121218, 121219],
           dtype='int64', length=51550)

In [25]:
masc_nulos_lon = geo_location_data["lon"].astype(str)  == "nan"
geo_location_data[masc_nulos_lon].index

Int64Index([    13,     14,     23,     42,     49,     62,     86,     98,
               104,    105,
            ...
            121208, 121210, 121211, 121212, 121213, 121214, 121215, 121216,
            121218, 121219],
           dtype='int64', length=51550)

In [26]:
#como lat y long tienen las mismas filas nan, escojo uno para obtener el indice
filas_nan= geo_location_data[masc_nulos_lon].index
print(geo_location_data.loc[filas_nan, ["state_name",'geonames_id','lat','lon']])


                          state_name  geonames_id  lat  lon
13                   Capital Federal    3430234.0  NaN  NaN
14                   Capital Federal    3430234.0  NaN  NaN
23          Bs.As. G.B.A. Zona Norte          NaN  NaN  NaN
42          Bs.As. G.B.A. Zona Norte    3429979.0  NaN  NaN
49          Bs.As. G.B.A. Zona Norte    3434130.0  NaN  NaN
...                              ...          ...  ...  ...
121214      Bs.As. G.B.A. Zona Norte    3428983.0  NaN  NaN
121215               Capital Federal    3436077.0  NaN  NaN
121216      Bs.As. G.B.A. Zona Norte    3436080.0  NaN  NaN
121218  Buenos Aires Costa Atlántica          NaN  NaN  NaN
121219               Capital Federal    3433955.0  NaN  NaN

[51550 rows x 4 columns]


In [40]:
df_geo_nulos = geo_location_data.loc[filas_nan, ["state_name",'geonames_id','lat','lon']]
#para tener una idea donde están concentrados los datos sin lat-lon 
df_geo_nulos["state_name"].shape

(51550,)

In [28]:
#saco los nulos de geonames_id
df_geo_nulos.dropna(subset=['geonames_id'], inplace=True)
df_geo_nulos.isnull().sum()

#y lat y lon nuevas que tengo que buscar para completar para completar

state_name         0
geonames_id        0
lat            43365
lon            43365
dtype: int64

Trabajando con la data externa de geonames 

In [29]:
geonames = pd.read_csv("ar_copy.csv", sep='\t', header=None)
#como no tengo nombres en la columnas para mejorar la extraccion renombro las que me interesan
geonames.rename({0: 'geoname_oficial', 4:"lat_oficial", 5:"lon_oficial"}, axis=1, inplace=True)
geonames.head(4)

,geoname_oficial,1,2,3,lat_oficial,lon_oficial,6,7,8,9,10,11,12,13,14,15,16,17,18
0,3427200,Sierra del 15,Sierra del 15,NaN,-37.96613,-57.94670,T,HLL,AR,NaN,1.0,6357.0,NaN,NaN,0,NaN,200,America/Argentina/Buenos_Aires,2016-01-30
1,3427201,Puesto 1 Napoleón Cué,Puesto 1 Napoleon Cue,NaN,-28.63333,-56.11667,S,HSE,AR,NaN,6.0,18168.0,NaN,NaN,0,NaN,79,America/Argentina/Cordoba,2016-01-30
2,3427202,Cañada del Zorro,Canada del Zorro,NaN,-32.16667,-59.50000,H,STM,AR,NaN,8.0,NaN,NaN,NaN,0,NaN,65,America/Argentina/Cordoba,1993-12-21
3,3427203,Estancia Zorraquín,Estancia Zorraquin,NaN,-31.51553,-58.55208,S,RNCH,AR,NaN,8.0,30113.0,NaN,NaN,0,NaN,71,America/Argentina/Cordoba,2016-01-30


In [30]:
#vemos que geoname_oficial es tipo int mientras geoname_id es float, por lo cual lo cambiamos
df_geo_nulos["geonames_id"] = df_geo_nulos["geonames_id"].astype(int)

In [31]:
#tambien vamos a sacar los nulos del geoname_oficial y por 
geonames.dropna(subset=['geoname_oficial'], inplace=True)
geonames[["geoname_oficial","lat_oficial","lon_oficial"]].isnull().sum().head(6)

geoname_oficial    0
lat_oficial        0
lon_oficial        0
dtype: int64

In [32]:
#ahora si vamos a buscar los datos de lat y lon desde el geoname_oficial del archivo geonames de internet
#Creamos un diccionario vacio para ubicar la Latitud
lat_dict = {}

#Creamos una tupla con los pares de key y value: usando un iterador de tuplas zip donde el primer
# elemento de cada iterador pasado se empareja con el primero del segundo y asi sucesivamente
geoname_lat = zip(geonames['geoname_oficial'], geonames['lat_oficial'])

#Rellenamos el diccionario
for geoname, lat_oficial in geoname_lat:
    lat_dict[geoname] = lat_oficial

In [33]:
#Creamos un diccionario para ubicar la Longitud
lon_dict = {}

#Creamos una tupla con los pares de key y value: usando un iterador de tuplas zip donde el primer
# elemento de cada iterador pasado se empareja con el primero del segundo y asi sucesivamente
geoname_lon = zip(geonames['geoname_oficial'], geonames['lon_oficial'])

#Rellenamos el diccionario
for geoname, lon_oficial in geoname_lon:
    lon_dict[geoname] = lon_oficial

In [34]:
df_geo_nulos["lat"] = df_geo_nulos['geonames_id'].map(lat_dict) 
df_geo_nulos["lon"] = df_geo_nulos['geonames_id'].map(lon_dict)

In [35]:
#cambio el nombre del data completado
df_geo_completo = df_geo_nulos
df_geo_completo

,state_name,geonames_id,lat,lon
13,Capital Federal,3430234,-34.58856,-58.43053
14,Capital Federal,3430234,-34.58856,-58.43053
42,Bs.As. G.B.A. Zona Norte,3429979,-34.41667,-58.83333
49,Bs.As. G.B.A. Zona Norte,3434130,-34.33333,-58.75000
62,Bs.As. G.B.A. Zona Sur,3436042,-34.83333,-58.16667
...,...,...,...,...
121213,Bs.As. G.B.A. Zona Norte,3430813,-34.49397,-58.51679
121214,Bs.As. G.B.A. Zona Norte,3428983,-34.49009,-58.53028
121215,Capital Federal,3436077,-34.56270,-58.45829
121216,Bs.As. G.B.A. Zona Norte,3436080,-34.46312,-58.53481


In [36]:
geo_location_data.isna().sum()

state_name         0
geonames_id    18717
lat-lon        51550
lat            51550
lon            51550
dtype: int64

In [37]:
df_geo_completo.isna().sum()

state_name       0
geonames_id      0
lat            161
lon            161
dtype: int64

In [38]:
porcentaje_completado_lat_lon = (df_geo_completo[["lat","lon"]].isna().sum() / geo_location_data[["lat","lon"]].isnull().sum())*100
porcentaje_completado_lat_lon

lat    0.312318
lon    0.312318
dtype: float64